In [ ]:
!pip install datasets
!pip install accelerate>=0.21.0
!pip install transformers[torch]
#!pip3 install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00


In [ ]:

!pip3 install pandas


In [ ]:
import pandas as pd


from datasets import load_dataset

dataset = load_dataset("finalgp3/dataset")
df = pd.DataFrame(dataset['train'])
print(df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

                                                data
0  [{'paragraphs': [{'context': 'Digital Image Pr...


In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['data'],
        num_rows: 1
    })
    test: Dataset({
        features: ['data'],
        num_rows: 1
    })
})


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_and_format(examples):
    tokenized_inputs = tokenizer(examples["content"], padding="max_length", truncation=True, max_length=1000)  # Reduced max_length
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_format, batched=True)


In [ ]:
# Assuming dataset is already loaded using load_dataset
def inspect_dataset(dataset_split):
    for i, item in enumerate(dataset_split):
        print(f"Item {i}: {item}")
        if i >= 2:  # Adjust this number to inspect more items if needed
            break

print("Inspecting Train Dataset:")
inspect_dataset(dataset['train'])

print("\nInspecting Test Dataset:")
inspect_dataset(dataset['test'])

Inspecting Train Dataset:
Item 0: {'data': [{'paragraphs': [{'context': 'Digital Image Processing Lecture 4: Filters (Part 2) & Edges and Contours Dr. Belhassen Akrout Computer Science Dept. What is anEdge? Edge? sharp change in brightness (discontinuities).', 'qas': [{'question': 'What is the main focus of Lecture 4 in Digital Image Processing?', 'id': 'unique_question_identifier_001', 'answers': [{'text': 'Filters (Part 2) & Edges and Contours', 'answer_start': 30}]}, {'question': 'Who is delivering Lecture 4 on Digital Image Processing?', 'id': 'unique_question_identifier_002', 'answers': [{'text': 'Dr. Belhassen Akrout', 'answer_start': 87}]}]}, {'context': 'Where do edges occur? Actual edges: Boundaries between objects Sharp change in brightness can also occur within object Reflectance changes Change in surface orientation Illumination changes.', 'qas': [{'question': 'Where do edges typically occur in image processing?', 'id': 'unique_question_identifier_003', 'answers': [{'text':

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Use the EOS token for padding

def process_data(dataset_split):
    tokenized_examples = []
    for entry in dataset_split:
        for item in entry['data']:
            for paragraph in item['paragraphs']:
                context = paragraph['context']
                for qa in paragraph['qas']:
                    question = qa['question']
                    answer = qa['answers'][0]['text']
                    input_text = f"question: {question} context: {context} answer: {answer}"
                    tokenized_input = tokenizer(input_text, truncation=True, padding='max_length', max_length=1024, return_tensors='pt')
                    tokenized_input['labels'] = tokenized_input.input_ids.clone()
                    tokenized_examples.append(tokenized_input)
    return tokenized_examples


# Assuming dataset is already loaded using load_dataset
train_data = process_data(dataset['train'])
test_data = process_data(dataset['test'])


In [ ]:
df = pd.DataFrame(test_data)
df.head()

,attention_mask,input_ids,labels
0,"[[tensor(1), tensor(1), tensor(1), tensor(1), ...","[[tensor(25652), tensor(25), tensor(1867), ten...","[[tensor(25652), tensor(25), tensor(1867), ten..."
1,"[[tensor(1), tensor(1), tensor(1), tensor(1), ...","[[tensor(25652), tensor(25), tensor(1867), ten...","[[tensor(25652), tensor(25), tensor(1867), ten..."
2,"[[tensor(1), tensor(1), tensor(1), tensor(1), ...","[[tensor(25652), tensor(25), tensor(1374), ten...","[[tensor(25652), tensor(25), tensor(1374), ten..."
3,"[[tensor(1), tensor(1), tensor(1), tensor(1), ...","[[tensor(25652), tensor(25), tensor(1867), ten...","[[tensor(25652), tensor(25), tensor(1867), ten..."
4,"[[tensor(1), tensor(1), tensor(1), tensor(1), ...","[[tensor(25652), tensor(25), tensor(1867), ten...","[[tensor(25652), tensor(25), tensor(1867), ten..."


In [ ]:
import torch
from torch.utils.data import Dataset

class QADataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        item = self.tokenized_data[idx]
        return {
            'input_ids': item['input_ids'].squeeze(),
            'attention_mask': item['attention_mask'].squeeze(),
            'labels': item['labels'].squeeze(),
        }


train_dataset = QADataset(train_data)
test_dataset = QADataset(test_data)


In [ ]:
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

In [1]:
# fine tuned it without comute metrics
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

model = GPT2LMHeadModel.from_pretrained("gpt2")
training_args = TrainingArguments(
    output_dir="./gpt2_finetuned_qa",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=5e-4  # Example: Increase the learning rate
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


KeyboardInterrupt: 

In [ ]:
trainer.train()

In [ ]:
# fine tuned it with compute metrics

from transformers import Trainer, TrainingArguments, GPT2LMHeadModel
from datasets import load_dataset, load_metric
import numpy as np

# Load your dataset and metric
metric = load_metric("accuracy")

model = GPT2LMHeadModel.from_pretrained("gpt2")

from datasets import load_metric
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metric = load_metric("accuracy")
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(
    output_dir="./gpt2_finetuned_qa",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=5e-4  # Example: Increase the learning rate
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics  # Pass the function reference here
)


trainer.train()


/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable

Step,Training Loss
10,1.661500
20,0.232200
30,0.226100
40,0.175700
50,0.187300
60,0.145600
70,0.093300
80,0.071200
90,0.094300
100,0.079100


TrainOutput(global_step=168, training_loss=0.1945010315449465, metrics={'train_runtime': 120.458, 'train_samples_per_second': 2.764, 'train_steps_per_second': 1.395, 'total_flos': 174020493312000.0, 'train_loss': 0.1945010315449465, 'epoch': 3.0})

In [ ]:
model.save_pretrained("/content/test_trainer")
tokenizer.save_pretrained("/content/test_trainer")

In [ ]:
# connect to google drive
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from google.colab import drive

drive.mount('/content/drive')




In [ ]:
#load any model from google drive
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_path = '/content/drive/MyDrive/Colab Notebooks/Models/FirstLLM'

# Load the language model
model = GPT2LMHeadModel.from_pretrained(model_path)

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_path)



In [ ]:
# save the model to google drive
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/Models/FirstLLM')  # Replace with your desired path
tokenizer.save_pretrained('/content/drive/MyDrive/Colab Notebooks/Models/FirstLLM')  # Replace with your desired path


In [ ]:
from transformers import GPT2LMHeadModel ,GPT2Tokenizer
model = GPT2LMHeadModel.from_pretrained("/content/test_trainer")
tokenizer = GPT2Tokenizer.from_pretrained("/content/test_trainer")
tokenizer.padding_side = "right"


In [ ]:
# Example settings
input_text = "What is the purpose of edge detection in image processing?"
temperature = 0.5 # higher values lead to more creative, less predictable text


In [ ]:
from transformers import GPT2LMHeadModel,AutoTokenizer

import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

      # Move the model to the chosen device


# Encode input text and create attention mask
input_ids = tokenizer.encode(input_text, return_tensors='pt') .to(device) # Move input_ids to the same device as the model
attention_mask = torch.ones(input_ids.shape, dtype=torch.long)  # Ensure attention_mask is on the same device
model.to(device)
# Set the do_sample parameter to True for using temperature
output = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    #max_length=200,
    max_new_tokens=30,
    temperature=temperature,
    do_sample=True,  # Enable sampling
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id,
)

# Decode generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True,pad_token="right")
print(generated_text)

In [ ]:
def generate_answer(question, temperature=1):
    # Tokenize the question with a prompt indicating that only an answer is expected
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

      # Move the model to the chosen device
    model.to(device)
    prompt = f"question: {question} \nanswer:"
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)


    # Generate a response with a specified temperature
    output = model.generate(input_ids, max_length=50, eos_token_id=tokenizer.eos_token_id, temperature=temperature)
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract only the answer part
    answer_start = response.find("answer: ") + len("answer: ")
    answer = response[answer_start:].strip()

    return answer

# Example usage
question = "What is the purpose of edge detection in image processing?"
answer = generate_answer(question, temperature=0.5)
print(answer)


In [ ]:
def generate_qa_summary(topic, temperature=1.0):
    # Check if GPU is available and set the device accordingly
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Move the model to the chosen device
    model.to(device)

    # Adjust the prompt to be more directive in generating QA pairs
    prompt = f"Create  quiz with question and answer about {topic}:\n\n"

    # Tokenize the prompt and move tensors to the same device as the model
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

    # Adjust generation settings for more dynamic output
    output = model.generate(
        max_length=1024,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        temperature=temperature,
        no_repeat_ngram_size=3,
        pad_token_id=tokenizer.eos_token_id,
        top_k=50,
        top_p=0.95,
        do_sample=True

    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # The response should contain a series of questions and answers
    return response

# Example usage
topic = " image processing"

print("summarization quiz with questions and answers about {topic}:\n\n")
for i in range(10):
    qa_summary = generate_qa_summary(topic, temperature=0.7)
    print(f'Question {i+1} {qa_summary}')


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


summarization quiz with questions and answers about {topic}:




A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Question 1 : What is the purpose of the Electronic Industry Citizenship Coalition (EICC)? context: Electronic Industry Citizens Coalition (EPICC): An organization promoting a common code of conduct for the electronics and ICT industry. Electronic Industry Assessment Tool (EAT): A system evaluating electronic products based on environmental criteria. answer: Evaluates electronic productsbased on environmental principles


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

Question 7 : What is the purpose of edge detection in image processing? context: Edge Detection Image processing task that finds edges and contours in images. Edges so important that human vision can reconstruct edge lines. answer: Finds edges and creours in image and compute edge lines


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Question 8 : What is the goal in creating an integrated linear filter in morphological operations? context: Finite Differences as Convolutions: Forward difference (right slope), Backward difference (left slope), Central Difference (average slope). Definition: Finites as Conventional Prewitt, Prewett, Prentice Hall. answer: Prewitting, Pending. answer? Forward difference, backward difference, Central Difference


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Question 9 : What is the concept of 'translation' in the context of morphological operations? context: Translation: A is set of pixels in binary image, B is an image. Note: An image is not digitally representable in the US. Translation: If we change B to dilation, we can reconstruct if B is a digitally representables. answer: Translation of A (black and white) is digitally representative of B, given the size of B and the type of B. answer is: The concept of applying a digitally representedable image
Question 10 : What is the purpose of the Digital Image Processing Lecture 5 in Digital Image processing? context: Digital Image Filter: A digital image processing task that finds edges and contours in images. Edges so important that human vision can reconstruct edge lines. answer: Finds edges and lines
